# Problema 4

Implementar el algoritmo Map-Reduce utilizando Python CUDA

In [1]:
# Comprobar que se este usando la GPU
!nvidia-smi

Tue Oct 28 04:31:47 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   46C    P8             12W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
# instalar Cupy
!pip install cupy-cuda12x

In [3]:
# =====================================================
# Map-Reduce numérico usando GPU (CuPy) vs CPU (NumPy)
# =====================================================
# Objetivo: agrupar (map) y reducir (sum y count) por clave (buckets).
# Ejecutar en Google Colab con GPU activada.
# =====================================================

import time
import numpy as np

# Intentaremos importar cupy; si no está instalado, descomenta la línea de instalación arriba.
try:
    import cupy as cp
except Exception as e:
    print("CuPy no está instalado o no se pudo importar. Instala cupy-cuda11x o cupy-cuda12x según tu CUDA.")
    raise e

# -------------- Parámetros y datos --------------
N = 50_000_000    # número de elementos
K = 1024          # número de claves/buckets (ej. 1024)
rng = np.random.default_rng(12345)

# Generar datos de entrada en CPU (float32)
# Usamos una distribución para que keys sean variadas
values_cpu = (rng.random(N).astype(np.float32) * 1e6)  # valores en rango [0,1e6)
# Definimos la función de mapa: key = int(value) % K  (ejemplo simple)
# Pero no calculamos keys aún (lo haremos en CPU y GPU según versión)

print(f"Datos generados: N={N}, K={K}")

# -------------- CPU (NumPy) - Map + Reduce --------------
def cpu_map_reduce(values, K):
    t0 = time.perf_counter()
    # Map: calcular claves en CPU
    keys = (values.astype(np.int64) % K)
    # Reduce: contamos ocurrencias por key y sumamos por key
    t_map = time.perf_counter()
    counts = np.bincount(keys, minlength=K)            # conteo por key
    sums = np.bincount(keys, weights=values, minlength=K)  # suma por key
    t1 = time.perf_counter()
    return {
        "counts": counts,
        "sums": sums,
        "t_map": t_map - t0,
        "t_reduce": t1 - t_map,
        "t_total": t1 - t0
    }

print("Ejecutando Map-Reduce en CPU (NumPy)... (esto puede tardar)")
cpu_res = cpu_map_reduce(values_cpu, K)
print(f"CPU total time: {cpu_res['t_total']:.4f} s, map: {cpu_res['t_map']:.4f}, reduce: {cpu_res['t_reduce']:.4f}")

# -------------- GPU (CuPy) - Map + Reduce --------------
# Idea: copiar datos a GPU, map keys en GPU, usar cupy.bincount (altamente optimizado) con weights
def gpu_map_reduce_cupy(values_cpu, K):
    # Transferir host -> device
    t0 = time.perf_counter()
    arr_gpu = cp.array(values_cpu)   # copia Host->Device
    cp.cuda.Stream.null.synchronize()
    t_after_copy = time.perf_counter()

    # Map (GPU): calcular keys
    # keys = arr_gpu.astype(int) % K  -> cast y mod en GPU
    keys_gpu = (arr_gpu.astype(cp.int64) % K)
    cp.cuda.Stream.null.synchronize()
    t_map_done = time.perf_counter()

    # Reduce (GPU): bincount para counts y sums (weights)
    counts_gpu = cp.bincount(keys_gpu, minlength=K)
    sums_gpu = cp.bincount(keys_gpu, weights=arr_gpu, minlength=K)
    cp.cuda.Stream.null.synchronize()
    t_reduce_done = time.perf_counter()

    # Traer resultados a host
    counts = counts_gpu.get()
    sums = sums_gpu.get()
    t_after_get = time.perf_counter()

    return {
        "counts": counts,
        "sums": sums,
        "t_copy_H2D": t_after_copy - t0,
        "t_map": t_map_done - t_after_copy,
        "t_reduce": t_reduce_done - t_map_done,
        "t_copy_D2H": t_after_get - t_reduce_done,
        "t_total": t_after_get - t0
    }

print("Ejecutando Map-Reduce en GPU (CuPy)... (warm-up y ejecución, esto puede tardar)")
# Warm-up (compilaciones / JIT internos)
_ = cp.array(np.array([1], dtype=np.float32))
cp.cuda.Stream.null.synchronize()

gpu_res = gpu_map_reduce_cupy(values_cpu, K)
print(f"GPU times (s): H2D={gpu_res['t_copy_H2D']:.4f}, map={gpu_res['t_map']:.4f}, reduce={gpu_res['t_reduce']:.4f}, D2H={gpu_res['t_copy_D2H']:.4f}, total={gpu_res['t_total']:.4f}")

# -------------- Verificación de igualdad (sanity check) --------------
print("Verificando que CPU y GPU obtuvieron el mismo resultado para algunos buckets...")
# Comparamos sums y counts para los primeros 10 buckets
for i in range(10):
    print(f"Bucket {i}: CPU count={cpu_res['counts'][i]}, GPU count={gpu_res['counts'][i]}, CPU sum={cpu_res['sums'][i]:.4f}, GPU sum={gpu_res['sums'][i]:.4f}")

# Chequeo numérico global (tolerancia pequeña)
counts_equal = np.array_equal(cpu_res['counts'], gpu_res['counts'])
sums_close = np.allclose(cpu_res['sums'], gpu_res['sums'], rtol=1e-6, atol=1e-3)
print(f"\nCounts equal: {counts_equal}, sums close: {sums_close}")

# -------------- Reporte final --------------
cpu_time = cpu_res['t_total']
gpu_time = gpu_res['t_total']
print(f"\n==== RESUMEN ====")
print(f"CPU total: {cpu_time:.4f} s")
print(f"GPU total: {gpu_time:.4f} s (incluye transferencias)")
print(f"Speedup (CPU/GPU): {cpu_time / gpu_time:.2f}x")


Datos generados: N=50000000, K=1024
Ejecutando Map-Reduce en CPU (NumPy)... (esto puede tardar)
CPU total time: 1.4388 s, map: 1.0440, reduce: 0.3948
Ejecutando Map-Reduce en GPU (CuPy)... (warm-up y ejecución, esto puede tardar)
GPU times (s): H2D=0.2091, map=0.6776, reduce=1.3413, D2H=0.0001, total=2.2281
Verificando que CPU y GPU obtuvieron el mismo resultado para algunos buckets...
Bucket 0: CPU count=49144, GPU count=49144, CPU sum=24630685729.1963, GPU sum=24630685729.1963
Bucket 1: CPU count=48687, GPU count=48687, CPU sum=24292767032.0323, GPU sum=24292767032.0323
Bucket 2: CPU count=49307, GPU count=49307, CPU sum=24632250741.3881, GPU sum=24632250741.3881
Bucket 3: CPU count=48642, GPU count=48642, CPU sum=24273659232.5328, GPU sum=24273659232.5328
Bucket 4: CPU count=49018, GPU count=49018, CPU sum=24555944273.2709, GPU sum=24555944273.2709
Bucket 5: CPU count=49156, GPU count=49156, CPU sum=24513925129.9876, GPU sum=24513925129.9876
Bucket 6: CPU count=48770, GPU count=4877

# Interpretacion de los resultados
* Datos: Procesaste 50 millones de números, agrupándolos en 1024 categorías.
* Resultados Correctos: La verificación (Counts equal: True, sums close: True) confirma que tanto la versión de CPU como la de GPU produjeron los mismos resultados correctos para el conteo y la suma por bucket. Esto es fundamental para validar el algoritmo en la GPU.
* Tiempos de CPU (NumPy):
        El tiempo total en CPU fue de 1.4388 segundos.
        El paso de "map" (calcular las claves) tomó la mayor parte del tiempo (1.0440 s), lo cual tiene sentido ya que involucra iterar sobre 50 millones de elementos y realizar operaciones aritméticas y un cast a entero.
        El paso de "reduce" (el bincount) fue más rápido (0.3948 s).
* Tiempos de GPU (CuPy):
        El tiempo total en GPU fue de 2.2281 segundos.
        Este tiempo incluye las transferencias de datos entre la CPU y la GPU.
        La transferencia de CPU a GPU (H2D) tomó 0.2091 s.
        El paso de "map" en GPU tomó 0.6776 s, que es significativamente más rápido que en la CPU (1.0440 s). Esto muestra el paralelismo de la GPU para esta operación.
        El paso de "reduce" en GPU (1.3413 s) fue más lento que en la CPU (0.3948 s). Esto es un poco inesperado y podría deberse a la implementación específica de cp.bincount para este tamaño de datos y número de buckets en tu GPU particular, o a gastos generales internos de CuPy/CUDA para esta operación.
        La transferencia de GPU a CPU (D2H) fue muy rápida (0.0001 s).
* Speedup (CPU/GPU): El resultado 0.65x significa que la versión de GPU fue más lenta que la versión de CPU en este caso particular (1.4388 / 2.2281 ≈ 0.65).

**Interpretación General:**

Aunque la GPU es muy buena para paralelizar operaciones, en este ejemplo específico, el beneficio de la ejecución paralela del map y el reduce en la GPU fue superado por el tiempo de transferencia de datos entre la CPU y la GPU (H2D) y, sorprendentemente, por el tiempo que tomó la operación de "reduce" en la GPU en comparación con la CPU.

Para que una implementación en GPU sea más rápida que en CPU, generalmente se necesita que:

    La operación a paralelizar sea computacionalmente intensiva y pueda dividirse en muchas tareas independientes que la GPU pueda ejecutar simultáneamente (el "map" lo logró).
    El tiempo de ejecución en la GPU sea significativamente menor que en la CPU.
    El tiempo ahorrado en la computación en la GPU sea mayor que el tiempo adicional gastado en transferir datos entre la CPU y la GPU.

En este caso, el "reduce" en la GPU fue más lento y la transferencia H2D añadió un costo que no se compensó con la aceleración del "map" y el "reduce" combinados en la GPU.

Posibles Razones del Resultado:

* Overhead de CuPy/CUDA: Para operaciones relativamente simples como bincount, el costo de configurar y ejecutar el kernel de CUDA a través de CuPy puede ser mayor que la ejecución altamente optimizada en NumPy para ciertos tamaños de datos.
* Características de la GPU: Aunque la Tesla T4 es una GPU potente, el rendimiento en operaciones específicas puede variar.
* Tamaño del problema vs. K: Con un K (número de buckets) relativamente pequeño (1024) comparado con N (50 millones), la operación bincount en NumPy ya es bastante eficiente. Si K fuera mucho más grande, o si la operación de "map" fuera mucho más compleja, la GPU podría mostrar una ventaja más clara.
* Transferencia de Datos: Para muchos problemas, la transferencia de datos entre CPU y GPU es un cuello de botella significativo. Si los datos ya estuvieran en la GPU (por ejemplo, si fueran el resultado de una operación GPU anterior), el tiempo total de la GPU sería t_map + t_reduce + t_copy_D2H, que en tu caso sería 0.6776 + 1.3413 + 0.0001 ≈ 2.019 s, aún más lento que la CPU.

En resumen, aunque se implemento correctamente el algoritmo Map-Reduce usando CuPy y se obtuvo los resultados correctos, para este problema específico con estos parámetros (N y K) y esta operación de "reduce", la implementación en CPU con NumPy resultó ser más rápida debido a la eficiencia del bincount en NumPy y al costo adicional de las transferencias de datos y el rendimiento del "reduce" en la GPU.

Es un excelente ejemplo que demuestra que la GPU no siempre es más rápida; depende de la naturaleza de la computación, el tamaño de los datos y el costo de la transferencia de datos.